In [1]:
import mechanicalsoup as ms
import pandas as pd
import numpy as np
import requests
import re
import os
import glob
import csv
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
import json
from bs4 import BeautifulSoup
import urllib.request as rq
import urllib
from zipfile import ZipFile
from io import BytesIO
import lxml
from sklearn import preprocessing
from sklearn import datasets, linear_model
from  sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.neural_network import MLPClassifier

In [10]:
def downloadhistoricaldata(trainQ, testQ, tables,session, flag):
    for link in tables:
        if(trainQ in link['href'] or testQ in link['href']):
            c = 'https://freddiemac.embs.com/FLoan/Data/' + link['href']
            response = session.get(c)
            z = ZipFile(BytesIO(response.content)) 
            z.extractall(os.getcwd())
            flag = 1
    return flag

In [11]:
def login(login, password, trainQ, testQ):
    
    
    s = requests.Session()
    url = "https://freddiemac.embs.com/FLoan/secure/auth.php"
    url2 = "https://freddiemac.embs.com/FLoan/Data/download.php"
    browser = ms.Browser(session = s)
    print("Logging in....")
    login_page = browser.get(url)
    login_form = login_page.soup.find("form",{"class":"form"})
    login_form.find("input", {"name":"username"})["value"] = login
    login_form.find("input", {"name":"password"})["value"] = password
    response = browser.submit(login_form, login_page.url)
    login_page2 = browser.get(url2)
    print("To the continue page...")

    next_form = login_page2.soup.find("form",{"class":"fmform"})
    a= next_form.find("input",{"name": "accept"}).attrs
    a['checked']=True

    response2 = browser.submit(next_form, login_page2.url)
    print("Start Downloading from..."+ response2.url)
    table = response2.soup.find("table",{"class":"table1"})
    
    t = table.find_all('a')
    flag = 0
    flag = downloadhistoricaldata(trainQ, testQ, t,s, flag) 

    if flag == 1:
        print("Data downloaded successfully!!")
    else:
        print("Error in downloading data")


In [12]:
login('komalambekar26@gmail.com','69dcTf=0','Q12005','Q22005')

Logging in....
To the continue page...
Start Downloading from...https://freddiemac.embs.com/FLoan/Data/download.php
Data downloaded successfully!!


In [2]:
trainQ = 'Q12005'
train_data = pd.read_csv(r"C:/Users/Komal/Downloads/historical_data_time_"+ trainQ + ".txt" , skipinitialspace=True, sep = "|" ,low_memory = False , header = None)

In [3]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,F105Q1000001,200504,190000.0,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F105Q1000001,200505,190000.0,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F105Q1000001,200506,190000.0,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F105Q1000001,200507,189000.0,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F105Q1000001,200508,189000.0,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
testQ = 'Q22005'
test_data = pd.read_csv(r"historical_data_time_"+ testQ + ".txt" , skipinitialspace=True, sep = "|" ,low_memory = False , header = None)

In [7]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,F105Q2000001,200507,214000.0,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F105Q2000001,200508,214000.0,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F105Q2000001,200509,214000.0,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F105Q2000001,200510,213000.0,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F105Q2000001,200511,213000.0,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
 train_data.columns=['loan_number','year-month', 'current_actual_upb', 'delq_status','loan_age', 'rem_months', 'repurchase_flag', 'modification_flag', 'zero_balance_code',
                      'zero_bal_date', 'current_int_rate', 'current_def_upb', 'ddlpi', 'mi_recoveries', 'net_sales_proceeds',
                      'non_mi_recoveries', 'expenses', 'legal_costs', 'maint_pres_costs', 'taxes_ins', 'misc_expenses',
                      'actual_loss_calc', 'modification_cost', 'stepmod_ind' , 'dpm_ind' , 'eltv' ]

In [9]:
test_data.columns=['loan_number','month', 'current_actual_upb', 'delq_status','loan_age', 'rem_months', 'repurchase_flag', 'modification_flag', 'zero_balance_code',
                      'zero_bal_date', 'current_int_rate', 'current_def_upb', 'ddlpi', 'mi_recoveries', 'net_sales_proceeds',
                      'non_mi_recoveries', 'expenses', 'legal_costs', 'maint_pres_costs', 'taxes_ins', 'misc_expenses',
                      'actual_loss_calc', 'modification_cost', 'stepmod_ind' , 'dpm_ind' , 'eltv' ]

In [10]:
train_data.head()

,loan_number,year-month,current_actual_upb,delq_status,loan_age,rem_months,repurchase_flag,modification_flag,zero_balance_code,zero_bal_date,...,expenses,legal_costs,maint_pres_costs,taxes_ins,misc_expenses,actual_loss_calc,modification_cost,stepmod_ind,dpm_ind,eltv
0,F105Q1000001,200504,190000.0,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F105Q1000001,200505,190000.0,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F105Q1000001,200506,190000.0,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F105Q1000001,200507,189000.0,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F105Q1000001,200508,189000.0,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train_data


,loan_number,year-month,current_actual_upb,delq_status,loan_age,rem_months,repurchase_flag,modification_flag,zero_balance_code,zero_bal_date,...,expenses,legal_costs,maint_pres_costs,taxes_ins,misc_expenses,actual_loss_calc,modification_cost,stepmod_ind,dpm_ind,eltv
0,F105Q1000001,200504,190000.00,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F105Q1000001,200505,190000.00,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F105Q1000001,200506,190000.00,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F105Q1000001,200507,189000.00,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F105Q1000001,200508,189000.00,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,F105Q1000001,200509,189000.00,0,5,355,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,F105Q1000001,200510,189000.00,0,6,354,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,F105Q1000001,200511,188557.98,0,7,353,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,F105Q1000001,200512,188348.10,0,8,352,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,F105Q1000001,200601,188348.10,0,9,351,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
test_data.head()

,loan_number,month,current_actual_upb,delq_status,loan_age,rem_months,repurchase_flag,modification_flag,zero_balance_code,zero_bal_date,...,expenses,legal_costs,maint_pres_costs,taxes_ins,misc_expenses,actual_loss_calc,modification_cost,stepmod_ind,dpm_ind,eltv
0,F105Q2000001,200507,214000.0,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F105Q2000001,200508,214000.0,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F105Q2000001,200509,214000.0,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F105Q2000001,200510,213000.0,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F105Q2000001,200511,213000.0,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def cleandf(df):
    df.delq_status = df.delq_status.replace('R', '1').astype('float64')
    df.rem_months = df.rem_months.replace(np.nan, 0)
    df.rem_months = df.rem_months.astype('category')
    df.repurchase_flag = df.repurchase_flag.replace(np.nan, 0)
    df.repurchase_flag = df.repurchase_flag.astype('category')
    df.modification_flag = df.modification_flag.replace(np.nan, 0)
    df.modification_flag = df.modification_flag.astype('category')
    df.zero_balance_code = df.zero_balance_code.replace(np.nan, 0)
    df.zero_balance_code = df.zero_balance_code.astype('category')
    df.zero_bal_date = df.zero_bal_date.replace(np.nan, 0)
    df.zero_bal_date = df.zero_bal_date.astype('category')
    df.current_def_upb = df.current_def_upb.replace(np.nan, 0)
    df.current_def_upb = df.current_def_upb.astype('category')
    df.ddlpi = df.ddlpi.replace(np.nan, 0)
    df.ddlpi = df.ddlpi.astype('category')
    df.mi_recoveries = df.mi_recoveries.replace(np.nan, 0)
    df.net_sales_proceeds = df.net_sales_proceeds.replace(np.nan, 0)
    df.net_sales_proceeds = df.net_sales_proceeds.replace('C', 1)
    df.net_sales_proceeds = df.net_sales_proceeds.replace('U', 0)
    df.net_sales_proceeds.astype('float64')
    df.non_mi_recoveries = df.non_mi_recoveries.replace(np.nan, 0)
    df.expenses = df.expenses.replace(np.nan, 0)
    df.legal_costs = df.legal_costs.replace(np.nan, 0)
    df.maint_pres_costs = df.maint_pres_costs.replace(np.nan, 0)
    df.taxes_ins = df.taxes_ins.replace(np.nan, 0)
    df.misc_expenses = df.misc_expenses.replace(np.nan, 0)
    df.actual_loss_calc = df.actual_loss_calc.replace(np.nan, 0)
    df.modification_cost = df.modification_cost.replace(np.nan, 0)

In [14]:
train_data_copy =train_data.copy(deep = True)

In [15]:
test_data_copy =test_data.copy(deep = True)

In [16]:
cleandf(train_data_copy)

In [17]:
cleandf(test_data_copy)

In [18]:
train_data_copy.head()

,loan_number,year-month,current_actual_upb,delq_status,loan_age,rem_months,repurchase_flag,modification_flag,zero_balance_code,zero_bal_date,...,expenses,legal_costs,maint_pres_costs,taxes_ins,misc_expenses,actual_loss_calc,modification_cost,stepmod_ind,dpm_ind,eltv
0,F105Q1000001,200504,190000.0,0.0,0,360,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,F105Q1000001,200505,190000.0,0.0,1,359,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,F105Q1000001,200506,190000.0,0.0,2,358,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,F105Q1000001,200507,189000.0,0.0,3,357,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,F105Q1000001,200508,189000.0,0.0,4,356,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [19]:
def statusDeliquent(row):
        if row['delq_status'] > 0:
            val = 1
        else:
            val = 0
        return val

In [20]:
train_data_copy['Deliquent'] = train_data_copy.apply(statusDeliquent, axis=1)

In [21]:
test_data_copy['Deliquent'] = test_data_copy.apply(statusDeliquent, axis=1)

In [22]:
train_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2403531 entries, 0 to 2403530
Data columns (total 27 columns):
loan_number           object
year-month            int64
current_actual_upb    float64
delq_status           float64
loan_age              int64
rem_months            category
repurchase_flag       category
modification_flag     category
zero_balance_code     category
zero_bal_date         category
current_int_rate      float64
current_def_upb       category
ddlpi                 category
mi_recoveries         float64
net_sales_proceeds    object
non_mi_recoveries     float64
expenses              float64
legal_costs           float64
maint_pres_costs      float64
taxes_ins             float64
misc_expenses         float64
actual_loss_calc      float64
modification_cost     float64
stepmod_ind           object
dpm_ind               object
eltv                  float64
Deliquent             int64
dtypes: category(7), float64(13), int64(3), object(4)
memory usage: 392.0+ MB


In [23]:
train_features = train_data_copy._get_numeric_data()

In [24]:
train_features.drop(['eltv'],axis =1, inplace = True)

In [25]:
test_features = test_data_copy._get_numeric_data()

In [26]:
test_features.drop(['eltv'],axis =1, inplace = True)

In [27]:
train_features.drop(['delq_status'],axis =1, inplace = True)

In [28]:
test_features.drop(['delq_status'],axis =1, inplace = True)

In [44]:
# To select the feature with elemination technique
def featureSelectionRFE(selector,data):
    decision = selector.support_
    important_feature = []
    columns = list(data)
    iter=0
    for col in columns:
        if decision[iter]:
            important_feature.append(col)
        iter+=1
    data_new = data.filter(important_feature,axis=1)
    return data_new

In [45]:
def confusionMatrixPrint(acc):
    print("")
    print("----------------------Confusion Matrix--------------------")
    print("                    Actual Result          ")
    print("                   0                  1          ")
    print("Expected  0        ",acc[0][0],"        ",acc[0][1])
    print("")
    print("result    1        ",acc[1][0],"        ",acc[1][1])
    print("")
    

In [49]:

traning_feature,traning_label,testing_feature,testing_label,testing_feture,testing_Label\
=dataSplit(Performance_data,'delinquent',year,Q1,Q2)
print("Traning",traning_feature.shape)
print("Testing",testing_feature.shape)

NameError: name 'dataSplit' is not defined

In [46]:
from sklearn.feature_selection import RFE
'''
This function builds a ligistic regression model using the features selected by **RFE** method
'''

def logistic_regressor(training_feature, traning_label, testing_feature, testing_label):
    logreg = linear_model.LogisticRegression(max_iter=20)
    selector = RFE(logreg, 5, step=1)
    selector.fit(training_feature, traning_label)
    testing_feature = featureSelectionRFE(selector, testing_feature)
    traning_feature = featureSelectionRFE(selector, training_feature)
    # we create an instance of Neighbours Classifier and fit the data.
    logreg.fit(traning_feature, training_label)
    print('Training results')
    r=logreg.predict(training_feature)
    cm=confusion_matrix(training_label,r)
    confusionMatrixPrint(cm)
    fpr, tpr, _ = roc_curve(training_label,r)
    result = np.sum(training_label.values.flatten() == r)/r.size
    print("Accuracy:",result*100)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
              label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    
    print ('\nTesting results')
    z=logreg.predict(testing_feature)
    cm=confusion_matrix(testing_label,z)
    confusionMatrixPrint(cm)
    fpr, tpr, _ = roc_curve(testing_label,z)
    result = np.sum(testing_label.values.flatten() == z)/z.size
    print("Accuracy:",result*100)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
              label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    

In [47]:
y = train_features.Deliquent

AttributeError: 'DataFrame' object has no attribute 'Deliquent'

In [33]:
y2 = test_features.Deliquent

In [34]:
train_features.drop(['Deliquent'],axis =1, inplace = True )

In [35]:
test_features.drop(['Deliquent'],axis =1, inplace = True )

In [48]:
logistic_regressor(train_features, y, test_features,y2)

NameError: name 'training_label' is not defined

In [40]:
from sklearn.ensemble import RandomForestClassifier
from  sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt


# Ramdom forest
def Random_forest_classification(training_feature, training_label, testing_feature, testing_label):
    
    # Creating the model
    rf = RandomForestClassifier(n_estimators=100,class_weight={0:1,1:0.001},n_jobs=-1)
    
    # Traning the model with traning data
    rf.fit(training_feature, training_label)
    
    print('Training Data')
    # Testing the model with the testing data
    r=rf.predict(training_feature)
    
    #Computing the confusion matrix
    cm=confusion_matrix(testing_label,r)
    confusionMatrixPrint(cm)
    
    result = np.sum(training_label.values.flatten() == r)/r.size
    print("Accuracy:",result*100)
    
    fpr, tpr, _ = roc_curve(training_label,r)

    plt.plot(fpr, tpr, color='darkorange',label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
    plt.plot([0, 0], [0, 1], color='darkorange')
    plt.xlim([-0.1, 1.0])
    plt.ylim([-0.1, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    Act_del = np.count_nonzero(training_label)
    Pred_del =  np.count_nonzero(z)
    length = r.size
    tp_del = cm[0][0]
    fp_del = cm[1][0]
    print('Number of records in dataset', length,'Actual delequents', Act_del, 'Predicted Delenquents',  Pred_del, 'Proper DElenquents', tp_del,'Improper delenqunets', fp_del)
    
    print('\nTesting Data')
    # Testing the model with the testing data
    z=rf.predict(testing_feature)
    
    #Computing the confusion matrix
    cm=confusion_matrix(testing_label,z)
    confusionMatrixPrint(cm)
    
    result = np.sum(testing_label.values.flatten() == z)/z.size
    print("Accuracy:",result*100)
    
    fpr, tpr, _ = roc_curve(testing_label,z)

    plt.plot(fpr, tpr, color='darkorange',label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
    plt.plot([0, 0], [0, 1], color='darkorange')
    plt.xlim([-0.1, 1.0])
    plt.ylim([-0.1, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    Act_del = np.count_nonzero(testing_label)
    Pred_del =  np.count_nonzero(z)
    length = z.size
    tp_del = cm[0][0]
    fp_del = cm[1][0]
    print('Number of records in dataset', length,'Actual delequents', Act_del, 'Predicted Delenquents',  Pred_del, 'Proper DElenquents', tp_del,'Improper delenqunets', fp_del)
    
    


C:\Users\Komal\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [41]:
Random_forest_classification(train_features, y, test_features,y2)

Training Data


ValueError: Found input variables with inconsistent numbers of samples: [5634100, 2403531]

In [42]:
def Neural_net(training_feature, training_label, testing_feature, testing_label):
    nn = MLPClassifier(solver='adam', alpha=1e-6,hidden_layer_sizes=(10, 2), random_state=3, max_iter=300,warm_start=True)

    # we create an instance of Neighbours Classifier and fit the data.
    nn.fit(training_feature, training_label)
    
    r=rf.predict(training_feature)
    
    #Computing the confusion matrix
    cm=confusion_matrix(testing_label,r)
    confusionMatrixPrint(cm)
    
    result = np.sum(training_label.values.flatten() == r)/r.size
    print("training Data")
    print("Accuracy:",result*100)
    
    fpr, tpr, _ = roc_curve(training_label,r)

    plt.plot(fpr, tpr, color='darkorange',label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
    plt.plot([0, 0], [0, 1], color='darkorange')
    plt.xlim([-0.1, 1.0])
    plt.ylim([-0.1, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    print('\nTesting Data')
    z=nn.predict(testing_feature)
    cm=confusion_matrix(testing_label,z)
    confusionMatrixPrint(cm)
    fpr, tpr, _ = roc_curve(testing_label,z)
    result = np.sum(testing_label.values.flatten() == z)/z.size
    print("Accuracy:",result*100)
    plt.plot(fpr, tpr, color='darkorange',
              label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    

In [43]:
Neural_net(train_features, y, test_features,y2)

NameError: name 'rf' is not defined